In [ ]:
import findspark
findspark.init("/usr/local/Cellar/apache-spark/3.1.2/libexec")

import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import*
from pyspark.sql.types import*
import time

import os

import numpy as np
import re
import pandas as pd 
import traceback
import sys
from time import sleep
import traceback
import smtplib
from collections import defaultdict

import ibm_db
import ibm_db_dbi
import ibm_db_sa
from datetime import datetime, date, time

In [ ]:
# In[ ]:


spark = SparkSession.builder.getOrCreate()


import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)
spark.sparkContext.setLogLevel("ERROR")

In [ ]:

######## Grab the Arguments #########

print("**Arg*****")
argslist = sys.argv
print("********")

In [ ]:
########## Function to Send Email #########

def sendmail(sub, body):
    try:
       SERVER = "us.relay.ibm.com"
       #SERVER = "smtp.gmail.com"
       #SERVER = smtplib.SMTP('smtp.gmail.com', 587)
       #SERVER.ehlo()
       FROM = "romdbload@radial.com"
       TO = ["mlakshm@us.ibm.com"] # must be a list
       SUBJECT = sub
       TEXT = body
       message = 'From:'+FROM+       '\nTo: '+", ".join(TO)+       '\nSubject: '+SUBJECT+'\n'+TEXT
       print(message)
       server = smtplib.SMTP(SERVER)
       server.sendmail(FROM, TO, message)
       server.quit()
       return 0
    except Exception as e:
       exp_tb=traceback.format_exc()
       print(exp_tb)



In [ ]:


tbllist=[]
jsonpathlists=[]
colslist=[]
json_exploded_path_lists=[]
col_len_lists=[]

with open('json_column_map_updated_newest.txt') as f:
    contents = f.read()
    contents=contents.strip()
    maplist=contents.split("TABLE_NAME:")
    #print(maplist)
    #maplist = list(filter([], maplist))
    for lmap in maplist:
        
        lmaplist=lmap.split("\n")
        #lmaplist = list(filter(None, lmaplist))
        tbl=lmaplist.pop(0)
        tbllist.append(tbl)
        collist=[]
        jsonpathlist=[]
        json_exploded_path_list=[]
        col_len_list=[]
        
        for col_json in lmaplist:
            coljsonarr=col_json.split(",")
            if (len(coljsonarr)==4):
               collist.append(coljsonarr[0])
               jsonpathlist.append(coljsonarr[1])
               json_exploded_path_list.append(coljsonarr[2])
               col_len_list.append(coljsonarr[3])
               
                
        colslist.append(collist)
        jsonpathlists.append(jsonpathlist)
        json_exploded_path_lists.append(json_exploded_path_list)
        col_len_lists.append(col_len_list)
            
            
f.close()


In [ ]:

tbllist = [x for x in tbllist if x != 'KAFKARADIAL.']
tbllist = [x for x in tbllist if x != '']
colslist = [x for x in colslist if x != []]
jsonpathlists = [x for x in jsonpathlists if x != []]
json_exploded_path_lists = [x for x in json_exploded_path_lists if x != []]
col_len_lists = [x for x in col_len_lists if x != []]


In [ ]:

print(tbllist)
print(colslist)
print(jsonpathlists)
print(json_exploded_path_lists)
print(col_len_lists)


In [ ]:

jdbcURL=""
user=""
password=""
jdbcstr=""
kafka_bootstrap_servers=""
scram_user=""
scram_pass=""
truststore_location=""
truststore_password=""

with open('cfg_info.txt') as f:
    
    contents = f.read()
    contents=contents.strip()
    cfg_infolist=contents.split("\n")
    jdbcURL=cfg_infolist[0]
    user=cfg_infolist[1]
    password=cfg_infolist[2]
    jdbcstr=cfg_infolist[3]
    kafka_bootstrap_servers=cfg_infolist[4]
    scram_user=cfg_infolist[5]
    scram_pass=cfg_infolist[6]
    truststore_location=cfg_infolist[7]
    truststore_password=cfg_infolist[8]
    kafkatopic=cfg_infolist[9]
    
f.close()    



In [ ]:
print(jdbcURL)
print(user)
print(password)
print(jdbcstr)
print(kafka_bootstrap_servers)
print(scram_user)
print(scram_pass)
print(truststore_location)
print(truststore_password)
print(kafkatopic)

In [ ]:
jsondf = spark.read.json("order_created_test3.json", multiLine=True)
order_created_schema=jsondf.schema

In [ ]:
jsondf.show()

In [ ]:
jsondf.printSchema()

In [ ]:
    
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
  .option("kafka.sasl.jaas.config","org.apache.kafka.common.security.scram.ScramLoginModule required username="+scram_user+" password="+scram_pass+";") \
  .option("kafka.security.protocol", "SASL_SSL") \
  .option("kafka.sasl.mechanism", "SCRAM-SHA-512") \
  .option("kafka.ssl.truststore.location",truststore_location) \
  .option("kafka.ssl.truststore.password", truststore_password) \
  .option("kafka.ssl.protocol", "TLSv1.2") \
  .option("kafka.ssl.enabled.protocols", "TLSv1.2") \
  .option("kafka.ssl.endpoint.identification.algorithm", "HTTPS") \
  .option("failOnDataLoss", "false") \
  .option("assign", """{"""+kafkatopic+"""}""") \
  .load() \
  .select(col("partition").cast("string"),col("offset").cast("string"),col("value").cast("string"),from_json(col("value").cast("string"), order_created_schema))
  

In [ ]:
df.printSchema()

In [ ]:
def writeToSQLWarehouse(jdf,epochId,tbl):
    
  print(tbl)
  #print(df)
  print("hgjghjgh")
  jdf.show()
  print("jhghjgjn")
  jdf.write.format("jdbc")   .mode("append")   .option("url", jdbcUrl)   .option("dbtable", tbl)   .option("user", user)   .option("password", password)   .save()


In [ ]:
arraysdf = pd.read_csv("arrays_order_created.txt", sep=",")

In [ ]:
def explodeDFTest(df):
    
    #df=df.withColumn("order", df.mvalue.order)
    #df=df.withColumn("customAttributes", df.mvalue.customAttributes)
    #df=df.withColumn("topicName", df.mvalue.topicName)
    #df=df.drop("mvalue")
    #df=df.drop("value")
    #df.show()
    
    for i, row in arraysdf.iterrows():
    
     try:
            
        json_path_to_explode=row['JSON_PATH_TO_EXPLODE']
        json_path_exploded=row['JSON_PATH_EXPLODED']
        print(json_path_to_explode)
        print(json_path_exploded)
    
        df=df.withColumn(json_path_exploded,explode_outer(json_path_to_explode))
        #df.show()
        
     except Exception as e:
        print(e)
        print(json_path_to_explode)
        
        
    return df



In [ ]:
def explodeDF(df):
    
    df=df.withColumn("order", df.mvalue.order)
    df=df.withColumn("customAttributes", df.mvalue.customAttributes)
    df=df.withColumn("topicName", df.mvalue.topicName)
    df=df.drop("mvalue")
    #df=df.drop("value")
    #df.show()
    
    for i, row in arraysdf.iterrows():
    
     try:
            
        json_path_to_explode=row['JSON_PATH_TO_EXPLODE']
        json_path_exploded=row['JSON_PATH_EXPLODED']
        print(json_path_to_explode)
        print(json_path_exploded)
    
        df=df.withColumn(json_path_exploded,explode_outer(json_path_to_explode))
        #df.show()
        
     except Exception as e:
        print(e)
        
        
    return df


In [ ]:
def connectToDB():
    #logger.info("Connecting to the Database....")
    print("Connecting to DB")
    conn=""
    try:
       array = { ibm_db.SQL_ATTR_AUTOCOMMIT : ibm_db.SQL_AUTOCOMMIT_OFF }
       #db2_conn = ibm_db.connect("DATABASE=BLUDB;HOSTNAME=10.5.197.84;PORT=30480;PROTOCOL=TCPIP;UID=admin;PWD=B09Y0kZgchfo;", "", "", array)
       #db2_conn = ibm_db.connect("DATABASE=BLUDB;HOSTNAME=9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud;PORT=32459;PROTOCOL=TCPIP;UID=hyf72343;PWD=oBc1ketTvsdM3kod;SECURITY=SSL", "", "", array)
       db2_conn = ibm_db.connect(jdbcstr, "", "", array)
       #db2_conn = ibm_db.connect("jdbc:db2://67beb010-4100-48d1-a281-7b5af97791aa.bs2ipa7w0uv9tsomi9ig.databases.appdomain.cloud:31815/bludb:sslConnection=true;", "a5406f1c", "eK1EhADAU7y3W61T")
       print(db2_conn)
    
       #conn = ibm_db_dbi.Connection(db2_conn)
       #conn.set_autocommit(False)
    except Exception as e:
       exp_tb=traceback.format_exc()
       print(exp_tb)
       sendmail("Error Retrieving Last Run Date",exp_tb)
       
    return db2_conn

In [ ]:
def popTablesBlkAtomicNew(df, epochId):  #### Bulk Insert
      
      
   #####  Create JDBC Connection #######
   print(datetime.now())
   conn=connectToDB()
      
   try:
      
    if df.rdd.isEmpty():
       print(df.rdd.isEmpty)
    else:
      #df.show()
      df=df.withColumnRenamed("from_json(CAST(value AS STRING))", "mvalue") 
      #df.select(col("mvalue").cast("string")).show()
      #valchk=NULL
      #dfnull=df.filter(col("mvalue").cast("string") == '{null, null, null}')
      #df=df.filter("mvalue is NOT NULL")
    
      #dfnull.show()
        
      #df=df.filter(col("mvalue").cast("string") != '{null, null, null}')
    
      #if dfnull.rdd.isEmpty():
            
      #      print("Empty")
            
      #else:
            
      #      nulldflist=dfnull.select("value").collect()
            
      #      for elm in nulldflist:
      #          flist=[]
      #          fval=elm[0]
      #          fparam=tuple([fval])
      #          fqry="INSERT INTO KAFKARADIAL.INSERT_FAILED_MSG ( msgvalue ) VALUES (?)"
      #          flist.append([fqry,fparam])
      #          insertData(flist,conn)
    
    
     
    
      if df.rdd.isEmpty():
       print(df.rdd.isEmpty)
    
      else:
    
         ###### Exploding Dataframe #######
         print("Starting to Explode Data:........")
         print(datetime.now()) 
         dfblk=explodeDF(df) 
         print("Finished Exploding Data:........")
         print(datetime.now()) 
      
          
         ##### Prepare Blk Dataframe & Insert Queries  #######
          
         #qlist,finaldflist=prepareData(dfblk,json_exploded_path_lists,colslist,epochId,tbllist)
         #qlist,finaldflist,dictlist,colslistup=prepareData(dfblk,json_exploded_path_lists,colslist,col_len_lists,epochId,tbllist)
         print("Starting to Prepare Data:........")
         print(datetime.now())  
         qlist=prepareData(dfblk,json_exploded_path_lists,colslist,col_len_lists,epochId,tbllist)
         print(datetime.now())
         print("Finishing to Prepare Data:.......")
    
         #print(qlist)
         print("*****")
         #print(len(finaldflist))   
            
         ###### Insert to Tables #######
         try:
             print("Starting to Insert:........")
             print(datetime.now())   
             insertData(qlist,conn)
             print(datetime.now())
             print("Finishing to Insert:........")
        
         except Exception as e:
                
             print(e)
             exp_tb=traceback.format_exc()
             print(exp_tb)
                
             #performSingleInserts(finaldflist,dictlist,colslistup,epochId,tbllist,conn)
            
    
   except Exception as e:
          print("here...........")
          print(e)
          exp_tb=traceback.format_exc()
          print(exp_tb)
          #popTablesAtomicUpdated(df, epochId, conn)
          
   finally:
         
          if conn==True:
            #print(conn)
            ibm_db.close(conn)

In [ ]:
def split_list(alist, wanted_parts):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

In [ ]:
   
def dropDupesBadDF(dfbadlists):
    
   ##### Prepare Bad Data Frame ######
   if len(dfbadlists) > 0:
       if len(dfbadlists[0]) > 0:
          dfb = dfbadlists[0][0]
   ctr=0
   for dfbadlist in dfbadlists:
   
     
     if len(dfbadlist) > 0:
         
       if ctr==0:
        #dfb=dfb.select("ORDER_ID","COLUMN_NAME","COLUMN_VALUE","MSG_PARTITION","MSG_OFFSET","MSG_VALUE")
        for dfbd in dfbadlist[1:]:
           #dfbd = dfbd.select("ORDER_ID","COLUMN_NAME","COLUMN_VALUE","MSG_PARTITION","MSG_OFFSET","MSG_VALUE")
              #dfb.show()
              print("^^^^")
              #dfbd.show()
              dfb=dfb.union(dfbd)
           
       else:
           
           for dfbd in dfbadlist:
           #dfbd = dfbd.select("ORDER_ID","COLUMN_NAME","COLUMN_VALUE","MSG_PARTITION","MSG_OFFSET","MSG_VALUE")
               #dfb.show()
               print("%%%%")
               #dfbd.show()
               dfb=dfb.union(dfbd)
       ctr=ctr+1
   if len(dfbadlists) > 0:
       if len(dfbadlists[0]) > 0: 
           dfb=dfb.dropDuplicates()
           print("**************")
           dfb.show()
           print("**************")
   

In [ ]:

def prepareData(dfblk,json_exploded_path_lists,colslist,col_len_lists,epochId,tbllist):
    
  
  qlist=[]  
  fullrowslist=[]
    
  dictlist=[]
  colslistup=[]
  badorders=[]
  badcolumns=[]
  column_names_list=[]
  column_names_str_list=[]
  
  print("Starting to Detect Bad Orders:........")
  print(datetime.now())  


  bad_order_qry_str = " INSERT INTO KAFKARADIAL.INSERT_FAILED_MSG ( ORDER_ID,COLUMN_NAME, COLUMN_VALUE, MSG_PARTITION, MSG_OFFSET, MSG_VALUE ) VALUES "
  badordervalues=[]
  bctr=0
  badqlist=[]

  for json_exploded_path_list,romlistfinal1,rom_col_len_list, tbl in zip(json_exploded_path_lists,colslist,col_len_lists,tbllist):
    
       
    finaldf,dfbadlist,col_len_list=prepareDFXA(dfblk,json_exploded_path_list,romlistfinal1,rom_col_len_list,epochId,tbl) 

    print("Starting to Collect Dataframe:........")
    print(datetime.now()) 

    finaldf.persist()
    print("Completed caching...")
    print(datetime.now()) 
    column_names = finaldf.columns
    column_names_str=','.join(column_names)
    column_names_list.append(column_names)
    print("Completed retrieving Columns...")
    print(datetime.now()) 
    #tbl_pkeys=pk_dict.get(tbl)
    #finaldf.dropDuplicates(tbl_pkeys)
    print("Completed Dropping Duplicates...")
    print(datetime.now()) 
    fullrows = finaldf.collect()  
    print("Completed collecting dataframe...")
    print(datetime.now()) 
    finaldf.unpersist()
    print("Completed unpersisting...")
    print(datetime.now()) 
    
    print("Ending to Format Data :........")
    print(datetime.now())
    
    
    print("Deduplication Begin:........")
    print(datetime.now())
    fullrows = list(set(fullrows))
    print("Deduplication End:........")
    print(datetime.now())
    
    fullrowslist.append(fullrows)
    
    
    print("Starting to prepare rows...")
    print(datetime.now())
    
    print("$$$$$$$$$$$$$$$$$$$$")
    print(tbl)
    print(len(fullrows))
    print(finaldf.count())
    print("$$$$$$$$$$$$$$$$$$$$")
    
    
    
    for row in fullrows:
            
            
          if row:
            badordinslist=[]
            orderid=row["ORDER_ID"]
            #msgvalue=row["MSG_VALUE"]
            #msgpartition=row["MSG_PARTITION"]
            #msgoffset=row["MSG_OFFSET"]
            if orderid == '':
                badorders.append(orderid)
                badordinslist.append("?")
                badordinslist.append("?")
                badordinslist.append("?")
                #badordinslist.append("?")
                #badordinslist.append("?")
                #badordinslist.append("?")
                badordervalues.append(orderid)
                badordervalues.append('')
                badordervalues.append('')
                #badordervalues.append(msgpartition)
                #badordervalues.append(msgoffset)
                #badordervalues.append(msgvalue)
                
            #msgvalue=row["VALUE"]
            for column,collen in zip(column_names,col_len_list):
                colval=row[column]
                #print(collen, colval)
                if( (collen.strip() != 'INTEGER') and (collen != 'NUMERIC (15,2)') and (collen != 'NUMERIC (14,4)') ):
                  if len(str(colval)) > int(collen):
                    badorders.append(orderid)
                    badordinslist.append("?")
                    badordinslist.append("?")
                    badordinslist.append("?")
                    #badordinslist.append("?")
                    #badordinslist.append("?")
                    #badordinslist.append("?")
                    badordervalues.append(orderid)
                    badordervalues.append(column)
                    badordervalues.append(str(colval))
                    #badordervalues.append(msgpartition)
                    #badordervalues.append(msgoffset)
                    #badordervalues.append(msgvalue)
                    
                elif (collen == 'NUMERIC(15,2)' ):
                    if len(str(colval)) > 17:
                        badorders.append(orderid)
                        badordinslist.append("?")
                        badordinslist.append("?")
                        badordinslist.append("?")
                        #badordinslist.append("?")
                        #badordinslist.append("?")
                        #badordinslist.append("?")
                        badordervalues.append(orderid)
                        badordervalues.append(column)
                        badordervalues.append(str(colval))
                        #badordervalues.append(msgpartition)
                        #badordervalues.append(msgoffset)
                        #badordervalues.append(msgvalue)
                        
                elif (collen == 'NUMERIC(14,4)') :
                    if len(str(colval)) > 18:
                        badorders.append(orderid)
                        badordinslist.append("?")
                        badordinslist.append("?")
                        badordinslist.append("?")
                        #badordinslist.append("?")
                        #badordinslist.append("?")
                        #badordinslist.append("?")
                        badordervalues.append(orderid)
                        badordervalues.append(column)
                        badordervalues.append(str(colval))
                        #badordervalues.append(msgpartition)
                        #badordervalues.append(msgoffset)
                       # badordervalues.append(msgvalue)
            
            if len(badordinslist) > 0:
            
                strbad = ','.join(badordinslist)
                strbad='('+strbad+')'
            
            
            
                if bctr==0:
              
                    bad_order_qry_str=bad_order_qry_str+strbad
                    bctr=bctr+1
                else:
                    bad_order_qry_str=bad_order_qry_str+" UNION ALL \n VALUES "+strbad
              
  print("&&&&&&&&&&&&")
  print(len(badorders))
    
  if len(badorders) > 0:  
        print("Entering&&&&&&&&&")                
        badparamtup=tuple(badordervalues)
        badqlist.append([bad_order_qry_str,badparamtup])  
        print(bad_order_qry_str)
        #print(len(badordervalues))
        #print(len(badparamtup))
        
        conn2=connectToDB()      
        insertData(badqlist,conn2)
        ibm_db.close(conn2)
        
        
        
                    
  print("Bad Orders:********************")  
  badorders=list(set(badorders))
  print(len(badorders))
  print(badorders)
  
  print("Finished Detecting Bad Orders:........")
  print(datetime.now()) 
    
  print("Starting Actual Data Preparation :........")
  print(datetime.now()) 
    
  for fullrows,column_names,romlistfinal1,rom_col_len_list,tbl in zip(fullrowslist,column_names_list,colslist,col_len_lists,tbllist):
    
    colslistup.append(column_names)
    column_names_str=','.join(column_names)
    
    qry_str="INSERT INTO "+tbl+" ("+column_names_str+") VALUES "
    listval=[]
       
    
    lenarr = len(fullrows)
    lenmid = lenarr//2
    
    rowsfirst = fullrows[:lenmid]
    rowssecond = fullrows[lenmid:]
    
    splitrowslist=[]
    splitrowslist.append(rowsfirst)
    splitrowslist.append(rowssecond) 
    
  
        
    splitrowslist=split_list(fullrows,  200)
    splitrowslist = [x for x in splitrowslist if x != []]
    
    # if tbl == 'KAFKARADIAL.ROM_ORDER_LINE_STG0':
    #      splitrowslist=split_list(fullrows,  50)
    #      splitrowslist = [x for x in splitrowslist if x != []]
    
    print("TBL:**********************************")
    print(tbl)
    print(len(splitrowslist))
    
    
    for splitrows in splitrowslist:
        
        
        paramlist=[]
        qry_str="INSERT INTO "+tbl+" ("+column_names_str+") VALUES "
        ctr=0
        
        
        for row in splitrows:
          #print(row) 
            
          if row:
            
            
           ordid=row["ORDER_ID"]
           if ordid not in badorders:
            list2=[]
            paramlist2=[]
            
            for column in column_names:
                colval=row[column]
                if colval == False:
                    colval="0"
                elif colval == True:
                    colval="1"
                paramlist.append(colval)
                list2.append("?")
                paramlist2.append(colval)
                
            str1 = ','.join(list2)
            str1='('+str1+')'
            
            #listval.append(str1)
            #listvalstr=','.join(listval)
            if ctr==0:
              
              qry_str=qry_str+str1
              ctr=ctr+1
            else:
              qry_str=qry_str+" UNION ALL \n VALUES "+str1
            #paramlist.append(tuple(list1))
        paramtup=tuple(paramlist)
        qlist.append([qry_str,paramtup])
            
        
  return qlist

  print("Ending Actual Data Preparation :........")
  print(datetime.now()) 


# In[ ]:



In [ ]:
    
def insertFailedBatch(df,epochId):
    
    tbl="KAFKARADIAL.INSERT_FAILED_BATCH"
    
    fdf=df.select("order.id","partition","offset","value")
    fdf=fdf.withColumnRenamed("id", "ORDER_ID")
    fdf=fdf.withColumnRenamed("partition", "MSG_PARTITION") 
    fdf=fdf.withColumnRenamed("offset", "MSG_OFFSET") 
    fdf=fdf.withColumnRenamed("value", "MSG_VALUE") 
    
    writeToSQLWarehouse(fdf, epochId, tbl)



In [ ]:
    
def insertBatch(df,epochId):
    
    tbl="KAFKARADIAL.INSERT_BATCH"
    
    fdf=df.select("order.id","partition","offset","value")
    fdf=fdf.withColumnRenamed("id", "ORDER_ID")
    fdf=fdf.withColumnRenamed("partition", "MSG_PARTITION") 
    fdf=fdf.withColumnRenamed("offset", "MSG_OFFSET") 
    fdf=fdf.withColumnRenamed("value", "MSG_VALUE") 
    
    writeToSQLWarehouse(fdf, epochId, tbl)



In [ ]:
def insertData(qlist,conn):
    
    
    for q in qlist:
        qry=q[0]
        param=q[1]
        stmt = ibm_db.prepare(conn, qry)
        print("here***")
        print(qry.splitlines()[0])
        #print(type(param))
        #print(param)
        print(datetime.now()) 
        ibm_db.execute(stmt, param)
        print(datetime.now()) 
        print("here####")
    ibm_db.commit(conn)

In [ ]:

def insSingleData(dictlist,tbllist,colslist,conn):
    
  print(datetime.datetime.now())
  #finaldford=finaldflist[0].groupBy("ORDER_ID").count()
  finaldfordlist=dictlist[0].keys()
  print(len(finaldfordlist))
        
  qry=""
  param=""
    
    
  for orderidrow in finaldfordlist:   
    
        
    try:
        
        
      qlist=[]   
    
      for dictelm,cols,tbl in zip(dictlist,colslist,tbllist):
        
        
          column_names_str=','.join(cols)
        
          paramlist=[]
          qry_str="INSERT INTO "+tbl+" ("+column_names_str+") VALUES "
          ctr=0
    
        
          rowsordlistnew = dictelm[orderidrow]
          #print(len(rowsordlistnew))
          
          for row in rowsordlistnew:
            
            #print(row)
            
            list2=[]
            str1=row[0]
            parampartlist=row[1]
            paramlist=paramlist+parampartlist
            
            if ctr==0:
                
              qry_str=qry_str+str1
              ctr=ctr+1
                
            else:
                
              qry_str=qry_str+" UNION ALL \n VALUES "+str1
            
            #paramlist.append(tuple(list1))
            
          paramtup=tuple(paramlist)
          qlist.append([qry_str,paramtup])
            
    
      insertSingleData(qlist,conn)    
        
    except Exception as e:
            
             print("hereëee....")
             exp_tb=traceback.format_exc()
             print(exp_tb)
             ibm_db.rollback(conn)



In [ ]:

def explodeDFJSON(df):
  
      #df=df.withColumn("order", df.value.order)
      #df=df.withColumn("customAttributes", df.value.customAttributes)
      #df=df.withColumn("topicName", df.value.topicName)
      #df=df.drop("value")
      #df.show()
    
      
      for i, row in arraysdf.iterrows():
      
          json_path_to_explode=row['JSON_PATH_TO_EXPLODE']
          json_path_exploded=row['JSON_PATH_EXPLODED']
          print(json_path_to_explode)
          print(json_path_exploded)
      
          df=df.withColumn(json_path_exploded,explode_outer(json_path_to_explode))
          df.show()
          
      return df



In [ ]:
def updateDF(sdf,tbl):
    
    #sdf.printSchema()
    if tbl=="KAFKARADIAL.ROM_ORDER_LINE_STG0":
       try:
        sdf=sdf.withColumn("IS_ASSOCIATE_DELIVERY", when(col("IS_ASSOCIATE_DELIVERY") == "false",lit("0") )        .when(col("IS_ASSOCIATE_DELIVERY") == "true",lit("1"))         .otherwise(col("IS_ASSOCIATE_DELIVERY")))
       except Exception as e:
         print(e)
    
    return sdf


In [ ]:


pk_dict = {'ROM_ORDER_HEADER_STG0': ['ORDER_ID'],
           
           'ROM_ORDER_LINE_STG0': ['ORDER_ID', 'ORDER_LINE_ID'],
           
           'ROM_RELATED_ORDERS_STG0' : ['ORDER_ID', 'RELATED_ORDER_ID'],
           
           'ROM_ORDER_PAYMENT_STG0' : ['ORDER_ID', 'TENDER_TYPE', 'PAYMENT_ACCT_NO'],
           
           'ROM_ORDER_PROMOTION_STG0' : ['ORDER_ID', 'ORDER_LINE_ID', 'ORDER_DISCOUNT_ID'],
           
           'ROM_ORDER_REFERENCES_STG0' : ['ORDER_ID', 'REFERENCE_TYPE', 'ATTRIB_NAME', 'ATTRIB_VALUE'],
           
           'ROM_ORDER_TAX_BREAKUP_STG0' : ['ORDER_ID', 'ORDER_LINE_ID', 'ORDER_SUB_LINE_ID', 
                                           'ORDER_CHARGE_ID', 'ORDER_DISCOUNT_ID'],
           
           'ROM_ORDER_LINE_RELATIONSHIP_STG0' : ['ORDER_ID', 'RELATION_TYPE', 
                                                'PARENT_LINE_ID', 'CHILD_LINE_ID'],
           
           'ROM_ORDER_CUSTOMER_INFO_STG0' : ['ORDER_ID', 'CUSTOMER_INFO_ID'],
           
           'ROM_ORDER_LINE_CHARGES_STG0' : ['ORDER_ID', 'ORDER_LINE_ID', 'ORDER_SUB_LINE_ID', 
                                            'ORDER_CHARGE_ID', 'ORDER_DISCOUNT_ID'],
           
           'ROM_ORDER_LINE_STATUS_STG0' : ['ORDER_ID', 'ORDER_LINE_ID', 'ORDER_SUB_LINE_ID',
                                           'STATUS_QTY', 'STATUS_ID']
    
}



In [ ]:

v=pk_dict.get('ROM_ORDER_HEADER_STG0')
type(v)


In [ ]:
jsondf.show()

In [ ]:

def updateDFSplFields_ROM_ORDER_HEADER_STG0(df, jsonpathlist_clean, rom_list_clean, romlen_list_clean):
    
    
        try:
        
            df = df.withColumn("ORDER_HEADER_KEY", concat(col("order.sellerId"), lit("-"), col("order.id")))
            jsonpathlist_clean.append("ORDER_HEADER_KEY")
            rom_list_clean.append("ORDER_HEADER_KEY")
            romlen_list_clean.append("1000000")
            
        except Exception as e:
            
             print(" Can't obtain ORDER_HEADER_KEY ")
             print(e)
        
        
        try: 
    
           df.select("order.hfrNumber")
           #df=df.withColumn("HFR_NUMBER" , col("order.hfrNumber") )
           jsonpathlist_clean.append("order.hfrNumber")
           rom_list_clean.append("HFR_NUMBER")
           romlen_list_clean.append("100000") 

        except Exception as e:
    
            try:
        
                df.select("order.raNumber")
                #df=df.withColumn("HFR_NUMBER" , col("order.raNumber") )
                jsonpathlist_clean.append("order.raNumber")
                rom_list_clean.append("HFR_NUMBER")
                romlen_list_clean.append("100000")
        
            except Exception as e:
        
                print(" HFR NUMBER order.hfrNumber or order.raNumber NOT PRESENT ")
            
            
        try: 
    
           df.select("order.billingAddressRefId")
           #df=df.withColumn("BILL_TO_KEY" , col("order.billingAddressRefId") )
           jsonpathlist_clean.append("order.billingAddressRefId")
           rom_list_clean.append("BILL_TO_KEY")
           romlen_list_clean.append("100000") 

        except Exception as e:
        
            print(" BILL_TO_KEY -->  order.billingAddressRefId NOT PRESENT ")
            
            
        return df, jsonpathlist_clean, rom_list_clean,romlen_list_clean
    
    

In [ ]:

def updateDFSplFields_ROM_ORDER_LINE_STG0(df, jsonpathlist_clean,rom_list_clean,romlen_list_clean ):
    
        try: 
            
           df2 = df.groupBy(col("order.id"), col("order_lineItems_sublineItems.quantity")).sum("order_lineItems_sublineItems.quantity")
           df2 = df2.withColumnRenamed("sum(order_lineItems_sublineItems.quantity AS `quantity`)", "ORDERED_QTY") 
           
           df=df.alias('a').join(df2.alias('b'),col('b.id') == col('a.order.id')).select("a.*","b.ORDERED_QTY")

            
           jsonpathlist_clean.append("ORDERED_QTY")
           rom_list_clean.append("ORDERED_QTY")
           romlen_list_clean.append("100000") 
            
        except Exception as e:
            print(e)
            try:
                
                df = df.groupBy(col("order.id"), col("order_lineItems.quantity")).sum("order_lineItems.quantity")
                df = df.withColumnRenamed("sum(order_lineItems.quantity AS `quantity`)", "ORDERED_QTY") 
                jsonpathlist_clean.append("ORDERED_QTY")
                rom_list_clean.append("ORDERED_QTY")
                romlen_list_clean.append("10000")
                
            except Exception as e:    
                print(e)
                print(" ORDERED_QTY not present ")
                
                 
        try: 
           print("********")
           #df.show()
           df.select("order_lineItems.shippingAddressRefId")
           #df=df.withColumn("SHIP_TO_KEY" , col("order_lineItems.shippingAddressRefId") )
           jsonpathlist_clean.append("order_lineItems.shippingAddressRefId")
           rom_list_clean.append("SHIP_TO_KEY")
           romlen_list_clean.append("100000") 

        except Exception as e:
            print(e)
            print(" SHIP_TO_KEY -->  order_lineItems.shippingAddressRefId NOT PRESENT ")
            
    
        return df, jsonpathlist_clean,rom_list_clean,romlen_list_clean
        



In [ ]:

def updateDFSplFields_ROM_RELATED_ORDERS_STG0( df,jsonpathlist_clean,rom_list_clean,romlen_list_clean ):

    try: 
            df.select("order.id","order.sellerId","order_relatedOrders.id","order_relatedOrders_relatedLines.lineNo","order_relatedOrders_relatedLines.originalLineNo" )
            df = df.withColumn("RELATED_ORDER_KEY", concat(col("order.id"), lit("-"), col("order.sellerId"), 
                 lit("-"),  col("order_relatedOrders.id"),  lit("-"), col("order_relatedOrders_relatedLines.lineNo"),
                 lit("-"),  col("order_relatedOrders_relatedLines.originalLineNo")    ))
            
            jsonpathlist_clean.append("RELATED_ORDER_KEY")
            rom_list_clean.append("RELATED_ORDER_KEY")
            romlen_list_clean.append("100000")
            
    except Exception as e:    
                jsonpathlist_clean=[]
                rom_list_clean=[] 
                romlen_list_clean=[]
                print(" RELATED_ORDER_KEY not present ")
                
                
    return df,jsonpathlist_clean,rom_list_clean,romlen_list_clean



In [ ]:
def updateDFSplFields_ROM_ORDER_PAYMENT_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean):


         try:
            
            
            df.select("order_paymentMethods.code","order_paymentMethods.creditCardNumber")     
            df=df.withColumn("PAYMENT_ACCT_NO", when(col("order_paymentMethods.code") == "CREDIT_CARD",col("order_paymentMethods.creditCardNumber") )      
                     .otherwise(None))
            
         except Exception as e:
            
            print(e)

         try:
            
            
            df.select("order_paymentMethods.code","order_paymentMethods.payPalNumber")     
            df=df.withColumn("PAYMENT_ACCT_NO", when(col("order_paymentMethods.code") == "PAYPAL",col("order_paymentMethods.payPalNumber") )      
                     .otherwise(None))
            
         except Exception as e:
            
            print(e)
            
            
         try:
            
            
            df.select("order_paymentMethods.code","order_paymentMethods.storedValueCardNumber")     
            df=df.withColumn("PAYMENT_ACCT_NO", when(col("order_paymentMethods.code") == "STORED_VALUE_CARD",col("order_paymentMethods.storedValueCardNumber") )      
                     .otherwise(None))
            
         except Exception as e:
            
            print(e)
            
            
         try:
            
            
            df.select("order_paymentMethods.code","order_paymentMethods.storedValueCardNumber")     
            df=df.withColumn("PAYMENT_ACCT_NO", when(col("order_paymentMethods.code") == "PREPAID_CARD",None))
            
         except Exception as e:
            
            print(e)
        
        
        
        #try:
            
            
            #df = df.withColumn("PAYMENT_ACCT_NO", when(col("order_paymentMethods.code") == "CREDIT_CARD",col("order_paymentMethods.creditCardNumber") )      
                     #.when(col("order_paymentMethods.code") == "PAYPAL",col("order_paymentMethods.payPalNumber")) 
                     #.when(col("order_paymentMethods.code") == "STORED_VALUE_CARD",col("order_paymentMethods.storedValueCardNumber"))
                     #.when(col("order_paymentMethods.code") == "PREPAID_CARD",None )
                     #.otherwise(None))
            
        #except Exception as e:
            
            #print("PAYMENT_ACCT_NO not found")


         try:
        
           ## OrderId + tenderType + AccountNo  ## AccountNo - verify Mapping 
           df = df.withColumn("PAYMENT_KEY", concat_ws('',col("order.id"), lit("-"), col("order_paymentMethods.tenderType"), 
                            lit("-"),  col("PAYMENT_ACCT_NO")))
                            
           jsonpathlist_clean.append("PAYMENT_KEY")
           rom_list_clean.append("PAYMENT_KEY")
           romlen_list_clean.append("100000")  
           
           #df.select("order_paymentMethods.code","PAYMENT_KEY").show() 
         except Exception as e:
              
            print("PAYMENT_KEY not found")
            print(e)   
                                                    
                                                    
                                                    
         return df, jsonpathlist_clean,rom_list_clean,romlen_list_clean
                                     

In [ ]:
def updateDFSplFields_ROM_ORDER_PROMOTION_STG0( df,jsonpathlist_clean,rom_list_clean,romlen_list_clean ):


       try:   
            
           df.select("order.id","order_lineItems.lineNo","order_lineItems_sublineItems.sublineNo","order_lineItems_charges.id","order_lineItems_charges_discounts.id")                                          
           ### OrderId + LineNo + SubLineNo + ChargeId + DiscountsId
           df = df.withColumn("PROMOTION_KEY", concat(col("order.id"), lit("-"), col("order_lineItems.lineNo"), 
                 lit("-"),  col("order_lineItems_sublineItems.sublineNo"),  lit("-"), col("order_lineItems_charges.id"),
                 lit("-"),  col("order_lineItems_charges_discounts.id")    ))  
                                                    
           jsonpathlist_clean.append("PROMOTION_KEY")
           rom_list_clean.append("PROMOTION_KEY")    
           romlen_list_clean.append("100000") 
                                                    
       except Exception as e:
            jsonpathlist_clean=[]
            rom_list_clean=[] 
            print(e) 
            print("PROMOTION_KEY not found")
            
                                   
       #print(jsonpathlist_clean)  
    
                                                    
       if "PROMOTION_KEY" in jsonpathlist_clean: 

        
        try: 
            
           df.select("*","order_lineItems_charges_discounts_customAttributes.name","order_lineItems_charges_discounts_customAttributes.value") 
           df=df.select("*","order_lineItems_charges_discounts_customAttributes.name",col("order_lineItems_charges_discounts_customAttributes.value").alias("cvalue"))
           df=df.withColumnRenamed("name", "PROMO_CUSTOM_NAME")
           df=df.withColumnRenamed("cvalue", "PROMO_CUSTOM_VALUE")
           df.show()
                                                    
           try:
            
                df = df.withColumn(  "PROMOTION_ID", 
                         when( col("PROMO_CUSTOM_NAME") == "promoold", col("PROMO_CUSTOM_VALUE") )
                       .otherwise(None) )
                jsonpathlist_clean.append("PROMOTION_ID")
                rom_list_clean.append("PROMOTION_ID")    
                romlen_list_clean.append("100000") 
            
           except Exception as e:
               
               print("PROMOTION_ID not retrieved")
               #print(e)
                                                    
           try:
            
            
                df = df.withColumn(  "PROMOTION_DESCRIPTION", 
                         when( col("PROMO_CUSTOM_NAME") == "promoDescription", col("PROMO_CUSTOM_VALUE") ) 
                        .otherwise(None) )
                jsonpathlist_clean.append("PROMOTION_DESCRIPTION")
                rom_list_clean.append("PROMOTION_DESCRIPTION")    
                romlen_list_clean.append("100000") 
            
           except Exception as e:
               
               print("PROMOTION_DESCRIPTION not retrieved")
               #print(e)
                                                    
                                                    
           try:
            
            
                df = df.withColumn(  "PROMOTION_CODE", 
                         when( col("PROMO_CUSTOM_NAME") == "promoCode", col("PROMO_CUSTOM_VALUE") ) 
                        .otherwise(None) )
                jsonpathlist_clean.append("PROMOTION_CODE")
                rom_list_clean.append("PROMOTION_CODE")    
                romlen_list_clean.append("100000") 
            
           except Exception as e:
               
               print("PROMOTION_CODE not retrieved")
               #print(e)
                            
           try:
            
            
                df = df.withColumn(  "PROMO_EFFECT_TYPE", 
                         when( col("PROMO_EFFECT_TYPE") == "EffectType", col("PROMO_CUSTOM_VALUE") ) 
                        .otherwise(None) )
                jsonpathlist_clean.append("PROMO_EFFECT_TYPE")
                rom_list_clean.append("PROMO_EFFECT_TYPE")    
                romlen_list_clean.append("100000") 
            
           except Exception as e:
               
               print("PROMO_EFFECT_TYPE not retrieved")
               #print(e)                                         
                                                    
        except Exception as e:
               
               print(" Can't retrieve Promotion Custom Attributes")
               #print(e)
                                                    

                    
       return df,jsonpathlist_clean,rom_list_clean,romlen_list_clean       
    
    
    

In [ ]:
def updateDFSplFields_ROM_ORDER_REFERENCES_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean):
    
    
           dft = spark.createDataFrame([], StructType([]))
           dft1 = spark.createDataFrame([], StructType([]))
           dft2 = spark.createDataFrame([], StructType([]))
           dft3 = spark.createDataFrame([], StructType([]))
           dft4 = spark.createDataFrame([], StructType([]))
            
           dfarr=[]
    
           try:

               
               dfexp = df.withColumn("order_customAttributes",explode("order.customAttributes"))
                
               if dfexp.rdd.isEmpty():
                
                   print(dfexp.rdd.isEmpty)
                    
               else:
                
                   dft=df.withColumn("REFERENCE_TYPE", lit("ORDER"))
                   dft = dft.withColumn("REFERENCE_KEY", col("order.id"))
                   dft = dft.withColumn("ATTRIB_NAME", col("order_customAttributes.name"))
                   dft = dft.withColumn("ATTRIB_VALUE", col("order_customAttributes.value"))
                   dft = dft.drop("order_customAttributes")
                   dfarr.append(dft)
                    
                   print("AT ORDER")
                
                
                                                    
           except Exception as e:
                                                    
                  print(e)  
                              
                        
           try:
            
               dfexp = df.withColumn("order_charges_customAttributes",explode("order_charges.customAttributes"))
                
               if dfexp.rdd.isEmpty():
                
                   print(dfexp.rdd.isEmpty)
                    
               else:
                
                   dft1=dfexp.withColumn("REFERENCE_TYPE", lit("CHARGES"))
                   dft1 = dft1.withColumn("REFERENCE_KEY", col("order.id"))
                   dft1 = dft1.withColumn("ATTRIB_NAME", col("order_charges_customAttributes.name"))
                   dft1 = dft1.withColumn("ATTRIB_VALUE", col("order_charges_customAttributes.value")) 
                   dft1 = dft1.drop("order_charges_customAttributes")
                   dfarr.append(dft1)
                    
                   print("AT CHARGES")
                                                                    
           except Exception as e:
                  
                  print("Can't find CHARGES")
                  #print(e)
                                                    
           try:
                                                    
               dfexp = df.withColumn("order_lineItems_customAttributes",explode("order_lineItems.customAttributes"))
                
               if dfexp.rdd.isEmpty():
                
                   print(dfexp.rdd.isEmpty)
                    
               else:
                
                   dft2=dfexp.withColumn("REFERENCE_TYPE", lit("LINE"))
                   dft2 = dft2.withColumn("REFERENCE_KEY", col("order.id"))
                   dft2 = dft2.withColumn("ATTRIB_NAME", col("order_lineItems_customAttributes.name"))
                   dft2 = dft2.withColumn("ATTRIB_VALUE", col("order_lineItems_customAttributes.value")) 
                   dft2 = dft2.drop("order_lineItems_customAttributes")
                   dfarr.append(dft2)
                   print("********")
                   print(dft2.columns) 
                   print("AT LINE")
                   print("********")
                
                                                    
           except Exception as e:
                  
                  print("Can't find LINE")
                  #print(e)
                                                    
            
           try:
            
               dfexp = df.withColumn("order_lineItems_charges_customAttributes",explode("order_lineItems_charges.customAttributes"))
            
               if dfexp.rdd.isEmpty():
                
                   print(dfexp.rdd.isEmpty)
                    
               else:
                
                   dft3=dfexp.withColumn("REFERENCE_TYPE", lit("LINE_CHARGES"))
                   dft3 = dft3.withColumn("REFERENCE_KEY", col("order.id"))
                   dft3 = dft3.withColumn("ATTRIB_NAME", col("order_lineItems_charges_customAttributes.name"))
                   dft3 = dft3.withColumn("ATTRIB_VALUE", col("order_lineItems_charges_customAttributes.value")) 
                   dft3 = dft3.drop("order_lineItems_charges_customAttributes")
                   dfarr.append(dft3)
                    
                   print("AT LINE_CHARGES") 
               
                                                    
           except Exception as e:
            
                  print("Can't find LINE_CHARGES")
                                                    
                  #print(e)
                                                    
            
           try:

               dfexp = df.withColumn("order_paymentMethods_customAttributes",explode("order_paymentMethods.customAttributes"))
            
               if dfexp.rdd.isEmpty():
                
                   print(dfexp.rdd.isEmpty)
                    
               else:
                
                   dft4=dfexp.withColumn("REFERENCE_TYPE", lit("PAYMENT"))
                   dft4 = dft4.withColumn("REFERENCE_KEY", col("order.id"))
                   dft3 = dft3.withColumn("ATTRIB_NAME", col("order_paymentMethods_customAttributes.name"))
                   dft3 = dft3.withColumn("ATTRIB_VALUE", col("order_paymentMethods_customAttributes.value")) 
                   dft4 = dft4.drop("order_paymentMethods_customAttributes")
                   dfarr.append(dft4)
                    
                   print("AT PAYMENT") 
                    
                                                    
           except Exception as e:
                  
                  print("Can't find PAYMENT")
                  #print(e)    
                    
                    
           try: 
            
               if dfarr:
              
                  for dfe in dfarr[1:]:
                    
                        dfe.select("REFERENCE_TYPE", "REFERENCE_KEY").show(2)
                        
                        print(dft.columns)
                        print("==========================")
                        print(dfe.columns)
                    
                        dft = dft.union(dfe)
                
                  df=dft
                    
                  jsonpathlist_clean.append("REFERENCE_KEY")
                  rom_list_clean.append("REFERENCE_KEY")    
                  romlen_list_clean.append("100000")   
                
                  jsonpathlist_clean.append("REFERENCE_TYPE")
                  rom_list_clean.append("REFERENCE_TYPE")    
                  romlen_list_clean.append("0")
                    
           except Exception as e:
                  
                  print("Error during union")                                  
                  print(e)
                    
                
           return df,jsonpathlist_clean,rom_list_clean,romlen_list_clean
                                                    


In [ ]:
def updateDFSplFields_ROM_ORDER_TAX_BREAKUP_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean):
    
    try:                                            
                                                    
           ### TAX_BREAKUP_KEY: OrderId + LineNo + SubLineNo + ChargeId + DiscountsId + TaxSequence 
           ### Need to check on Tax Sequence.   
           df.select("order.id","order_lineItems.lineNo","order_lineItems_sublineItems.sublineNo","order_lineItems_charges.id","order_lineItems_charges_discounts.id","TaxSequence")
           df = df.withColumn("TAX_BREAKUP_KEY", concat(col("order.id"), lit("-"), col("order_lineItems.lineNo"), 
                 lit("-"),  col("order_lineItems_sublineItems.sublineNo"),  lit("-"), col("order_lineItems_charges.id"),
                 lit("-"),  col("order_lineItems_charges_discounts.id"), lit("-"),col("TaxSequence")    ))  
                                                    
           jsonpathlist_clean.append("TAX_BREAKUP_KEY")
           rom_list_clean.append("TAX_BREAKUP_KEY")  
           romlen_list_clean.append("100000") 
                                                    
    except Exception as e:
              
            print("TAX_BREAKUP_KEY not found")
            #print(e)   
            
            
    return df,jsonpathlist_clean,rom_list_clean,romlen_list_clean

In [ ]:
def updateDFSplFields_ROM_ORDER_LINE_RELATIONSHIP_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean):
    
    try:                                                 
             
           ### RELATED_ORDER_KEY: SellerId + OrderId + RelType + ParentLineNo + ChileLineNo
           df.select("order.sellerId","order.id","order_relatedOrders.type","order_lineItems.lineNo","order_relatedOrders_relatedLines.lineNo")
           df = df.withColumn("RELATED_ORDER_KEY", concat(col("order.sellerId"), lit("-"), col("order.id"), 
                 lit("-"),  col("order_relatedOrders.type"),  lit("-"), col("order_lineItems.lineNo"),
                 lit("-"),  col("order_relatedOrders_relatedLines.lineNo") ))  
                                                    
           jsonpathlist_clean.append("RELATED_ORDER_KEY")
           rom_list_clean.append("RELATED_ORDER_KEY")    
           romlen_list_clean.append("100000") 
                                                    
    except Exception as e:
            jsonpathlist_clean = []
            rom_list_clean = []
            romlen_list_clean = []
            print("RELATED_ORDER_KEY not found")
            #print(e)  
            
    return df, jsonpathlist_clean,rom_list_clean,romlen_list_clean
    

In [ ]:
def updateDFSplFields_ROM_ORDER_LINE_CHARGES_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean):
    
           try:   
                                                    
            ### LINE_CHARGES_KEY:  OrderId + LineNo + SubLineNo + ChargeId + DiscountsId
            df.select("order.id","order_lineItems.lineNo","order_lineItems_sublineItems.sublineNo","order_lineItems_charges.id","order_lineItems_charges_discounts.id")
            
            df = df.withColumn("LINE_CHARGES_KEY", concat(col("order.id"), lit("-"),  col("order_lineItems.lineNo"),
                 lit("-"),  col("order_lineItems_sublineItems.sublineNo"),
                 lit("-"),  col("order_lineItems_charges.id"),
                 lit("-"),  col("order_lineItems_charges_discounts.id") ))  
                                                    
            jsonpathlist_clean.append("LINE_CHARGES_KEY")
            rom_list_clean.append("LINE_CHARGES_KEY")
            romlen_list_clean.append("0")
                                                    
           except Exception as e:
                                                    
                 print(e)    
                                                    
           
           try:
                
               df.withColumn("order_lineItems_charges_customAttributes", explode_outer("order_lineItems_charges.customAttributes"))  
               df=df.withColumn("order_lineItems_charges_customAttributes", explode_outer("order_lineItems_charges.customAttributes"))  
               df.select("*",
                         "order_lineItems_charges_customAttributes.name","order_lineItems_charges_customAttributes.value")
               df=df.select("*",
                    "order_lineItems_charges_customAttributes.name","order_lineItems_charges_customAttributes.value")
               df=df.withColumnRenamed("name","CHARGES_CUST_ATTR_NAME")  
               df=df.withColumnRenamed("value","CHARGES_CUST_ATTR_VALUE")  
              
               df=df.withColumn("REFERENCE", when(col("CHARGES_CUST_ATTR_NAME") == "Reference", col("CHARGES_CUST_ATTR_VALUE") ) )
               df=df.drop("CHARGES_CUST_ATTR_NAME")
               df=df.drop("CHARGES_CUST_ATTR_VALUE")
               
               jsonpathlist_clean.append("REFERENCE")
               rom_list_clean.append("REFERENCE")
               romlen_list_clean.append("100000") 
            
           except Exception as e:
                                                    
                 print(e)   
                                                    
           try:

               df.select("order_lineItems_charges.detail.originalChargeAmount")
               #df=df.withColumn("ORIGINAL_CHARGE_AMT", col("order_lineItems_charges.detail.originalChargeAmount") ) 
            
               jsonpathlist_clean.append("order_lineItems_charges.detail.originalChargeAmount")
               rom_list_clean.append("ORIGINAL_CHARGE_AMT")
               romlen_list_clean.append("100000") 
      
           except Exception as e:
                         
                 try:
                      
                     df.select("order_lineItems_charges.detail.amount")
                     #df=df.withColumn("ORIGINAL_CHARGE_AMT", col("order_lineItems_charges.detail.amount") ) 
                     jsonpathlist_clean.append("order_lineItems_charges.detail.amount")
                     rom_list_clean.append("ORIGINAL_CHARGE_AMT")
                     romlen_list_clean.append("100000") 

                 except Exception as e:
                           
                           print("ORIGINAL_CHARGE_AMT not found....")                         
                           print(e)
                                                    


           return df, jsonpathlist_clean,rom_list_clean,romlen_list_clean


In [ ]:
def updateDFSplFields_ROM_ORDER_LINE_STATUS_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean):

          df.select("order.sellerId", "order.id", "order_lineItems_sublineItems.sublineNo", 
                     "order_lineItems_charges.id", "order_lineItems_charges_discounts.id" )
          df = df.withColumn("ORDER_LINE_STATUS_KEY", concat(col("order.sellerId"), lit("-"),  col("order.id"),
                 lit("-"),  col("order_lineItems_sublineItems.sublineNo"),
                 lit("-"),  col("order_lineItems_charges.id"),
                 lit("-"),  col("order_lineItems_charges_discounts.id") ))                                           
                                                    
          jsonpathlist_clean.append("ORDER_LINE_STATUS_KEY")
          rom_list_clean.append("ORDER_LINE_STATUS_KEY") 
          romlen_list_clean.append("100000")  
            
            
          return df, jsonpathlist_clean,rom_list_clean,romlen_list_clean
    

In [ ]:
def updateDFSplFields(df,tbl,jsonpathlist_clean,rom_list_clean,romlen_list_clean):
    

    
    
    if ( tbl ==  "KAFKARADIAL.ROM_ORDER_HEADER_STG0" ):
        
       print("Accessing Special Fields from "+tbl) 
        
       df,jsonpathlist_clean,rom_list_clean,romlen_list_clean = updateDFSplFields_ROM_ORDER_HEADER_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean)
                    
        
    elif ( tbl == "KAFKARADIAL.ROM_ORDER_LINE_STG0" ):
        
        print("Accessing Special Fields from "+tbl) 
        
        df,jsonpathlist_clean,rom_list_clean,romlen_list_clean = updateDFSplFields_ROM_ORDER_LINE_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean)
        
        
    elif ( tbl == "KAFKARADIAL.ROM_RELATED_ORDERS_STG0" ):
        
        print("Accessing Special Fields from "+tbl) 
        
        df,jsonpathlist_clean,rom_list_clean,romlen_list_clean = updateDFSplFields_ROM_RELATED_ORDERS_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean)
                
        
    elif ( tbl == "KAFKARADIAL.ROM_ORDER_PAYMENT_STG0" ): 
        
        print("Accessing Special Fields from "+tbl) 
        
        df,jsonpathlist_clean,rom_list_clean,romlen_list_clean = updateDFSplFields_ROM_ORDER_PAYMENT_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean)                              
                                                         
    
    elif ( tbl == "KAFKARADIAL.ROM_ORDER_PROMOTION_STG0" ):  
        
        print("Accessing Special Fields from "+tbl) 
        
        df,jsonpathlist_clean,rom_list_clean,romlen_list_clean = updateDFSplFields_ROM_ORDER_PROMOTION_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean)
                                                    
    
    elif ( tbl == "KAFKARADIAL.ROM_ORDER_REFERENCES_STG0" ): 
        
        print("Accessing Special Fields from "+tbl) 
                                                    
        df,jsonpathlist_clean,rom_list_clean,romlen_list_clean = updateDFSplFields_ROM_ORDER_REFERENCES_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean)
                                                    
                                                 
    elif ( tbl == "KAFKARADIAL.ROM_ORDER_TAX_BREAKUP_STG0" ):
        
        print("Accessing Special Fields from "+tbl) 
                                                 
        df,jsonpathlist_clean,rom_list_clean,romlen_list_clean = updateDFSplFields_ROM_ORDER_TAX_BREAKUP_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean)                                        
                                                    
                                                 
    elif ( tbl == "KAFKARADIAL.ROM_ORDER_LINE_RELATIONSHIP_STG0" ):
        
         print("Accessing Special Fields from "+tbl) 
         
         df,jsonpathlist_clean,rom_list_clean,romlen_list_clean = updateDFSplFields_ROM_ORDER_LINE_RELATIONSHIP_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean)                                    
                                                 
    elif ( tbl == "KAFKARADIAL.ROM_ORDER_CUSTOMER_INFO_STG0" ):
        
            print("Accessing Special Fields from "+tbl) 
            print("No Special Fields")
                                                 
    elif  ( tbl == "KAFKARADIAL.ROM_ORDER_LINE_CHARGES_STG0" ):  
        
          print("Accessing Special Fields from "+tbl) 
                                                    
          df,jsonpathlist_clean,rom_list_clean,romlen_list_clean = updateDFSplFields_ROM_ORDER_LINE_CHARGES_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean)
                                                 
    elif  ( tbl == "KAFKARADIAL.ROM_ORDER_LINE_STATUS_STG0" ):
        
          print("Accessing Special Fields from "+tbl) 
                                                 
          df,jsonpathlist_clean,rom_list_clean,romlen_list_clean = updateDFSplFields_ROM_ORDER_LINE_STATUS_STG0(df,jsonpathlist_clean,rom_list_clean,romlen_list_clean)                                      
                                                 
                                                 
    return df,jsonpathlist_clean,rom_list_clean,romlen_list_clean     
    

In [ ]:
def prepareDFXA(df,json_exploded_path_list,romlistfinal1,rom_col_len_list,epochId,tbl):
    
       print("&&&&&&&&&&")
       #df.show()
       jsonpathlist_clean=[]
       rom_list_clean = [] 
       romlen_list_clean = []
       dfbadlist=[]
       
       #df.registerTempTable("Order_Created")
        
       for elm,colval,collen in zip(json_exploded_path_list,romlistfinal1,rom_col_len_list):
        
           try:
             df.select(elm)                 
             jsonpathlist_clean.append(elm)
             rom_list_clean.append(colval)  
             romlen_list_clean.append(collen)
           except Exception as e:
             #print(e)
             print(elm)
            
            
           
       try:
            df.select("topicName","partition","offset")
            df = df.withColumn("DW_SOURCE_ID" , concat(col("topicName"), lit("-"), lit(datetime.now()), lit("-"), col("partition"), lit("-"), col("offset") ))
            jsonpathlist_clean.append("DW_SOURCE_ID")
            rom_list_clean.append("DW_SOURCE_ID")
            romlen_list_clean.append("0")
        
       except Exception as e:
        
            print(" Can't obtain DW_SOURCE_ID ")
            print(e)
    
            
            
       df,jsonpathlist_clean,rom_list_clean,romlen_list_clean= updateDFSplFields(df,tbl,jsonpathlist_clean,rom_list_clean, romlen_list_clean)
             
             
       # for elm,colval,collen in zip(jsonpathlist_clean,rom_list_clean,romlen_list_clean):
        
       #     try:
               
       #       if (tbl=="KAFKARADIAL.ROM_ORDER_PAYMENT_STG0"):
       #           #df.show()
       #           #df.select(elm).show()
       #           #print(length(df.select(elm).collect()[0][0]))  
       #           print("here")
               
               
       #       if collen != "INTEGER":  
       #           #df=df.withColumn(colval, when(length(col(colval)) > collen,substring(col(colval), 1, collen )        .otherwise(col(colval))))
       #            df=df.withColumn(colval, when(length(col(colval).cast("string")) > collen,substring(col(colval).cast("string"), 1, collen )        .otherwise(col(colval))))

       #     except Exception as e:
       #       print(e)
       #       print(colval)
              
       
       cdf=df.select(jsonpathlist_clean)
       #cdf.show()
            
       sdf=cdf.toDF(*rom_list_clean)
       #sdf.show()
         
    
       ######## Update Dataframe ########
       print("Updating")
       sdf = updateDF(sdf,tbl)
       #sdf.show()
    
    
       ##### Remove Duplicates ########
       print("Dropping Duplicates")
       #tbl_pkeys=pk_dict.get(tbl)
       #inaldf=sdf.dropDuplicates(tbl_pkeys)
       finaldf=sdf
       
      
       #writeToSQLWarehouse(dfb,epochId,"INSERT_FAILED_MSG")
       
    
       #finaldf=sdf.dropDuplicates(["ORDER_ID"])
       #finaldf=sdf.withColumn("MERGEDCOL",hash(concat(*rom_list_clean)))
       #finaldf.show()
       #finaldf=finaldf.dropDuplicates(['MERGEDCOL'])
       #finaldf.show()
       #finaldf=finaldf.drop('MERGEDCOL')
       #finaldf.show()
       #finaldf=sdf.distinct()
       #finaldf=sdf.groupBy(rom_list_clean).count.sort().show
       #print(sdf)
       #finaldf.printSchema()
    
       return finaldf,dfbadlist,romlen_list_clean



In [ ]:
print(datetime.now())

In [ ]:

query1 = df     .writeStream     .outputMode("append")    .option("partition.assignment.strategy", "range")    .foreachBatch(popTablesBlkAtomicNew)    .start()


#query1 = df     .writeStream     .outputMode("append")    .option("checkpointLocation", "/myapp/GChkPt11")    .option("partition.assignment.strategy", "range")    .foreachBatch(popTablesBlkAtomicNew)    .start()
    
query1.awaitTermination(10)


In [ ]:
query1.status

In [ ]:
while ( (query1.status['isDataAvailable'] == True) and (query1.status['isTriggerActive'] == True) ):
    
    query1.status
    
query1.stop()